In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import *

airbnb_df = spark.table("az_adb_simbus_training.inside_air_bnb_project_schema.bronze_listings")


# Basic Preprocessing

In [0]:
airbnb_df = airbnb_df.drop('neighbourhood_group', 'license')

In [0]:
airbnb_df = airbnb_df.na.drop(how='any')

In [0]:
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType

whitelist_tokens = {
    "p/", "p/mo", "p/month", "p/wk", "p/week", "p/day", "p/sq ft",
    "sq ft", "sq. ft.", "sq m", "br", "ba", "w/d", "a/c", "hwd", "apt",
    "unit", "ft", "hr", "mo", "rm", "dep", "incl", "util", "pkg", "ref",
    "elec", "gas", "hwh"
}

non_alphanum_pattern = re.compile(r"[^a-zA-Z0-9/]")

def strict_clean_text(text):
    if text is None:
        return None

    text = re.sub(r"<.*?>", " ", text) #html tags

    tokens = text.split()
    cleaned_tokens = []

    for token in tokens:
        token_lower = token.lower()
        if token_lower in whitelist_tokens:
            cleaned_tokens.append(token)
        else:
            cleaned = re.sub(non_alphanum_pattern, " ", token)
            if cleaned:
                cleaned_tokens.append(cleaned)

    return " ".join(cleaned_tokens)

strict_clean_text_udf = udf(strict_clean_text, StringType())
airbnb_df = airbnb_df.withColumn("name_cleaned", strict_clean_text_udf(col("name")))


In [0]:
airbnb_df = airbnb_df.drop("name")

In [0]:
from pyspark.sql.functions import to_date

airbnb_df = airbnb_df.withColumn("last_review", to_date("last_review", "yyyy-MM-dd"))


In [0]:
airbnb_df.write.format("delta").mode("overwrite").saveAsTable("az_adb_simbus_training.inside_air_bnb_project_schema.silver_listing")


In [0]:
. Monthly Price Aggregates
Columns:
- listing_id
- neighborhood
- year
- month
- avg_price

In [0]:
Monthly Occupancy Rates
Columns:
- listing_id
- neighborhood
- year
- month
- occupancy_rate

In [0]:
Average Rating per Listing
Columns:
- listing_id
- avg_rating

In [0]:
4. Host Features
Columns:
- host_id
- is_superhost
- response_time
- total_listings
- avg_rating (from reviews)
- avg_price (from listings)
- commidites offer(hard to do)/text extraction